# Training using numpy arrays
In this notebook we'll take a look at how to use numpy data to train a neural network in CNTK.
The model used here is a basic classification model that uses random training samples. The goal is to demonstrate how to use numpy, so we left out the stuff that you don't need.

## Setting the random seed
The sample relies on random numbers. In order to keep the results predictable we'll set the random seed for the random number generators to a fixed value.

In [1]:
import cntk
import numpy 

cntk._cntk_py.set_fixed_random_seed(1337)
numpy.random.seed = 1337

## Generating sample data
Before we can actually train the model, we need to generate some sample data. We need a set of input features and a set of labels for our model. 

The labels are stored as one-hot vectors. To generate them we first create an identity matrix that has the same size as the number of classes that we want to predict. We can now use the `random.choice` function to select an index value at random. You can repeat this random selection process a number of times by specifying this as the second argument of the function. We feed the randomly selected indices to the indexer of the identity matrix. This will select random rows from the identity matrix 20.000 times.

In [2]:
import numpy as np

num_samples = 20000

y = np.eye(2)[np.random.choice(2,num_samples)].astype(np.float32)
X = np.random.random(size=(num_samples, 4)).astype(np.float32)

## Building the model
The model is a basic classification model. We use one hidden layer and an output layer. 
Both have a sigmoid activation function. 

In [3]:
from cntk.layers import Dense, Sequential
from cntk import input_variable, default_options
from cntk.ops import sigmoid

with default_options(activation=sigmoid):
    model = Sequential([
        Dense(4),
        Dense(2)
    ])

features = input_variable(4)

z = model(features)

## Training the neural network
To train the neural network you can invoke the `train` method on the loss function to feed the numpy data in chunks. These chunks are called minibatches. 

We've attached a progress printer to the training process so you can see what's going on.
Also note, that we used a `minibatch_size` setting to control the number of samples per minibatch.

In [4]:
from cntk.learners import sgd
from cntk.losses import binary_cross_entropy
from cntk.logging import ProgressPrinter

progress_writer = ProgressPrinter(0)

labels = input_variable(2)
learner = sgd(z.parameters, lr=0.5)
loss = binary_cross_entropy(z, labels)

training_summary = loss.train((X,y), 
  parameter_learners=[learner], 
  callbacks=[progress_writer], 
  minibatch_size=512)

 average      since    average      since      examples
    loss       last     metric       last              
 ------------------------------------------------------
Learning rate per minibatch: 0.5
      1.4        1.4          0          0           512
      1.4       1.39          0          0          1536
     1.39       1.39          0          0          3584
     1.39       1.39          0          0          7680
     1.39       1.39          0          0         15872
